In [1]:
import os
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
ayuntamientos = {'ABANILLA-JOG': ['ES330024_1', 'GASTOS DANA AYUNTAMIENTO DE ABANILLA', 'ES330024', "30"],
 'ABARÁN-EBB': ['ES330031_1', 'GASTOS DANA AYUNTAMIENTO DE ABARÁN', 'ES330031', "30"],
 'AENA-MAC-EBB': ['ES117023_1', 'GASTOS DANA AENA (MITMA)', 'ES117023', "28"],
 'AGULLENT-JOG': ['ES303050_1','GASTOS DANA AYUNTAMIENTO DE AGULLENT','ES303050', "46"],
 'ALBAIDA-MAC': ['ES346012_1', 'GASTOS DANA AYUNTAMIENTO DE ALBAIDA', 'ES346012', "46"],
 'ALBATERA-JOG': ['ES303044_1','GASTOS DANA AYUNTAMIENTO DE ALBATERA','ES303044', "03"],
 #'ALCAUCÍN-RENUNCIA': ['ES329032_1','GASTOS DANA AYUNTAMIENTO DE ALCAUCÍN','ES329032', "29"],
 'ALCÁZARES-MAC': ['ES330025_1','GASTOS DANA AYUNTAMIENTO DE ALCÁZARES, LOS','ES330025', "30"],
 'ALCÓNTAR-JOG': ['ES304029_1','GASTOS DANA AYUNTAMIENTO DE ALCÓNTAR', 'ES304029', "04"],
 'ALGORFA-JOG': ['ES303016_1', 'GASTOS DANA AYUNTAMIENTO DE ALGORFA', 'ES303016', "03"],
 'ALHAURÍN DE LA TORRE-MAC': ['ES329005_1','GASTOS DANA AYUNTAMIENTO DE ALHAURÍN DE LA TORRE','ES329005', "29"],
 'ALMANSA-EBB': ['ES302002_1', 'GASTOS DANA AYUNTAMIENTO DE ALMANSA', 'ES302002', "02"],
 'ALMERÍA-DGM': ['ES304004_1', 'GASTOS DANA AYUNTAMIENTO DE ALMERÍA', 'ES304004', "04"],
 'ALMORADÍ-DGM': ['ES303017_1','GASTOS DANA AYUNTAMIENTO DE ALMORADÍ','ES303017', "46"],
 "ALQUERIA DE AZNAR-EBB": ['ES303018_1', "GASTOS DANA AYUNTAMIENTO DE ALQUERIA D'ASNAR, L'", 'ES303018', "46"],
 'AP CARTAGENA-MAC': ['ES117025_1', 'GASTOS DANA AUTORIDAD PORTUARIA CARTAGENA (MITMA)', 'ES117025', "30"],
 'ARBOLEAS-MAC': ['ES304025_1','GASTOS DANA AYUNTAMIENTO DE ARBOLEAS','ES304025', "04"],
 'ARCHENA-MAC': ['ES330005_1', 'GASTOS DANA AYUNTAMIENTO DE ARCHENA', 'ES330005', "30"],
 'ARCHIDONA-EBB': ['ES329023_1','GASTOS DANA AYUNTAMIENTO DE ARCHIDONA','ES329023', "29"],
 'BAZA-MAC': ['ES318001_1', 'GASTOS DANA AYUNTAMIENTO DE BAZA', 'ES318001', "18"],
 'BENIEL-JOG': ['ES330013_1', 'GASTOS DANA AYUNTAMIENTO DE BENIEL', 'ES330013', "30"],
 #'BIENSERVIDA-RENUNCIA': [],
 'BIGASTRO-JOG': ['ES303051_1','GASTOS DANA AYUNTAMIENTO DE BIGASTRO', 'ES303051', "46"],
 'BLANCA-JOG': ['ES330015_1', 'GASTOS DANA AYUNTAMIENTO DE BLANCA', 'ES330015', "30"],
 'CAMPOS DEL RÍO - DGM': ['ES330030_1', 'GASTOS DANA AYUNTAMIENTO DE CAMPOS DEL RÍO', 'ES330030', "30"],
 #'CARTAGENA-JOG': ['ES3300065_1','GASTOS DANA AYUNTAMIENTO DE CARTAGENA', 'ES330006', "30"],
 'CASTILLA LA MANCHA- DGM': ['ES211001_1','GASTOS DANA COMUNIDAD AUTÓNOMA DE CASTILLA-LA MANCHA','ES211001', "45"],
 'CASTRIL-DGM': ['ES318054_1', 'GASTOS DANA AYUNTAMIENTO DE CASTRIL', 'ES318054', "18"],
 'CHAUCHINA-JOG': ['ES318053_1', 'GASTOS DANA AYUNTAMIENTO DE CHAUCHINA', 'ES318053', "18"],
 'CIEZA-JOG': ['ES330002_1', 'GASTOS DANA AYUNTAMIENTO DE CIEZA', 'ES330002', "30"],
 'COCENTAINA-EBB': ['ES303025_1', 'GASTOS DANA AYUNTAMIENTO DE COCENTAINA', 'ES303025', "03"],
 'CONF.HIDRO.JÚCAR-EBB': ['ES130001_1', 'GASTOS DANA  CONFEDERACION HIDROGRAFICA DEL JÚCAR, O.A. DEL MITECO', 'ES130001', "46"],
 'CONF.HIDRO.SEGURA-MAC': ['ES130006_1', 'GASTOS DANA CONFEDERACION HIDROGRAFICA DEL SEGURA, O.A. DEL MITECO', 'ES130006', "30"],
 #'COX-RENUNCIA': ['ES303053_1', 'GASTOS DANA AYUNTAMIENTO DE COX', 'ES303053', "03"],
 'COÍN-JOG': ['ES329029_1', 'GASTOS DANA AYUNTAMIENTO DE COÍN', 'ES329029', "29"],
 'CREVILLENT-MAC': ['ES303047_1', 'GASTOS DANA AYUNTAMIENTO DE CREVILLENT', 'ES303047', "03"],
 #'CUEVAS DEL CAMPO-JOG': ['ES318058_1', 'GASTOS DANA AYUNTAMIENTO DE CUEVAS DEL CAMPO', 'ES318058', "18"],
 #'CULLAR-RENUNCIA-JOG': ['ES318056_1','GASTOS DANA AYUNTAMIENTO DE CÚLLAR','ES318056', "18"],
 #'data_blp': [],
 'DAYA NUEVA-DGM': ['ES303046_1', 'GASTOS DANA AYUNTAMIENTO DE DAYA NUEVA', 'ES303046', "03"],
 'DENIA-MAC': ['ES303026_1', 'GASTOS DANA AYUNTAMIENTO DE DÉNIA', 'ES303026', "03"],
 'DGCMTM y AU-JOG': ['ES117007_1', 'GASTOS DANA DG CARRETERASV (MITMA)', 'ES117007', "30"],
 #'DP ALBACETE-DGM': ['ES302026_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE ALBACETE', 'ES302026', "02"],
 'DP ALMERÍA- DGM': ['ES304006_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE ALMERÍA', 'ES304006', "04"],
 'DP GRANADA-JOG': ['ES318007_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE GRANADA', 'ES318007', "18"],
 'DP MÁLAGA-JOG': ['ES329003_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE MÁLAGA', 'ES329003', "29"],
 'FUENTE-ÁLAMO-JOG': ['ES302052_1', 'GASTOS DANA AYUNTAMIENTO DE FUENTE-ÁLAMO', 'ES302052', "30"],
 'GALLARDOS-JOG': ['ES304026_1', 'GASTOS DANA AYUNTAMIENTO DE GALLARDOS, LOS', 'ES304026', "04"],
 'GENERALITAT VALENCIANA-EBB': ['ES209001_1', 'GASTOS DANA GENERALITAT VALENCIANA', 'ES209001', "46"],
 'GRANADA-JOG': ['ES318003_1', 'GASTOS DANA AYUNTAMIENTO DE GRANADA', 'ES318003', "18"],
 #'HELLÍN-DGM': ['ES302051_1', 'GASTOS DANA AYUNTAMIENTO DE HELLÍN', 'ES302051', "02"],
 #'HOYA GONZALO-JOG': ['ES302058_1', 'GASTOS DANA AYUNTAMIENTO DE HOYA-GONZALO', 'ES302058', "02"],
 'HUESCAR-DGM': ['ES318042_1', 'GASTOS DANA AYUNTAMIENTO DE HUÉSCAR', 'ES318042', "18"],
 'JACARILA-JOG': ['ES303052_1', 'GASTOS DANA AYUNTAMIENTO DE JACARILLA', 'ES303052', "03"],
 'JUNTA ANDALUCÍA-JOG': ['ES204001_1', 'GASTOS DANA JUNTA DE ANDALUCÍA', 'ES204001', "41"],
 'L-OLLERIA-JOG': ['ES346056_1', "GASTOS DANA AYUNTAMIENTO DE OLLERIA, L'", 'ES346056', "46"],
 'LOJA - JOG': ['ES318055_1', 'GASTOS DANA AYUNTAMIENTO DE LOJA', 'ES318055',"18"],
 #'LORCA-DGM': ['ES330028_1', 'GASTOS DANA AYUNTAMIENTO DE LORCA', 'ES330028', "30"],
 'LORCHA-DGM': ['ES303054_1', "GASTOS DANA AYUNTAMIENTO DE LORCHA/ORXA, L'", 'ES303054', "03"],
 'LORQUÍ-JOG': ['ES330017_1', 'GASTOS DANA AYUNTAMIENTO DE LORQUÍ', 'ES330017', "30"],
 'MINISTERIO DEFENSA-EBB': ['ES114007_1', 'GASTOS DANA MINISTERIO DE DEFENSA', 'ES114007', "28"],
 'MOLINA DEL SEGURA-MAC': ['ES330009_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINA DE SEGURA', 'ES330009', "30"],
 'MOLINICOS-DGM': ['ES302018_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINICOS', 'ES302018', "02"],
 'MONACHIL- DGM': ['ES318026_1', 'GASTOS DANA AYUNTAMIENTO DE MONACHIL', 'ES318026', "18"],
 'NERPIO-JOG': ['ES302023_1', 'GASTOS DANA AYUNTAMIENTO DE NERPIO', 'ES302023', "02"],
 'NIJAR-MAC': ['ES304007_1', 'GASTOS DANA AYUNTAMIENTO DE NÍJAR', 'ES304007', "04"],
 'NOVELDA-EBB': ['ES303045_1', 'GASTOS DANA AYUNTAMIENTO DE NOVELDA', 'ES303045', "03"],
 'OJÓS-JOG': ['ES330026_1', 'GASTOS DANA AYUNTAMIENTO DE OJÓS', 'ES330026', "30"],
 'OLULA DEL RÍO-JOG': ['ES304027_1', 'GASTOS DANA AYUNTAMIENTO DE OLULA DEL RÍO', 'ES304027', "04"],
 'ONTINYENT-MAC': ['ES346007_1', 'GASTOS DANA AYUNTAMIENTO DE ONTINYENT', 'ES346007', "46"],
 #'ONTUR-DGM': ['ES302053_1', 'GASTOS DANA AYUNTAMIENTO DE ONTUR', 'ES302053', "02"],
 'ORIHUELA-MAC': ['ES303003_1', 'GASTOS DANA AYUNTAMIENTO DE ORIHUELA', 'ES303003', "03"],
 'OTOS-JOG': ['ES346081_1', 'GASTOS DANA AYUNTAMIENTO DE OTOS', 'ES346081', "46"],
 'PARCENT-EBB': ['ES303049_1', 'GASTOS DANA AYUNTAMIENTO DE PARCENT', 'ES303049', "03"],
 'PILAR DE LA HORADADA EBB': ['ES303041_1', 'GASTOS DANA AYUNTAMIENTO DE PILAR DE LA HORADADA', 'ES303041', "03"],
 'PINOS GENIL-MAC': ['ES318057_1', 'GASTOS DANA AYUNTAMIENTO DE PINOS GENIL', 'ES318057', "18"],
 'PUEBLA DE DON FADRIQUE-EBB': ['ES318043_1', 'GASTOS DANA AYUNTAMIENTO DE PUEBLA DE DON FADRIQUE', 'ES318043', "18"],
 'PUERTO LUMBRERAS-JOG': ['ES330033_1', 'GASTOS DANA AYUNTAMIENTO DE PUERTO LUMBRERAS', 'ES330033', "30"],
 'PULPÍ-JOG': ['ES304016_1', 'GASTOS DANA AYUNTAMIENTO DE PULPÍ', 'ES304016', "04"],
 'PURCHENA-JOG': ['ES304019_1', 'GASTOS DANA AYUNTAMIENTO DE PURCHENA', 'ES304019', "04"],
 'REDOVÁN-DGM': ['ES303048_1', 'GASTOS DANA AYUNTAMIENTO DE REDOVÁN', 'ES303048', "03"],
 'REGIÓN DE MURCIA-DGM': ['ES208001_1', 'GASTOS DANA REGIÓN DE MURCIA', 'ES208001', "30"],
 'RENFE-MAC': ['ES117024_1', 'GASTOS DANA RENFE (MITMA)', 'ES117024', "28"],
 'RINCÓN DE LA VICTORIA-DGM': ['ES329011_1', 'GASTOS DANA AYUNTAMIENTO DE RINCÓN DE LA VICTORIA', 'ES329011', "29"],
 'RIOJA-JOG': ['ES304024_1', 'GASTOS DANA AYUNTAMIENTO DE RIOJA', 'ES304024',"04"],
 'SAN FULGENCIO-MAC': ['ES302057_1', 'GASTOS DANA AYUNTAMIENTO DE SAN FULGENCIO', 'ES302057', "03"],
 'SAN JAVIER-JOG': ['ES330001_1', 'GASTOS DANA AYUNTAMIENTO DE SAN JAVIER', 'ES330001', "30"],
 'SANTOMERA-JOG': ['ES330014_1', 'GASTOS DANA AYUNTAMIENTO DE SANTOMERA', 'ES330014', "30"],
 'SERÓN-JOG': ['ES304022_1', 'GASTOS DANA AYUNTAMIENTO DE SERÓN', 'ES304022', "04"],
 'TABERNO - JOG': ['ES304030_1', 'GASTOS DANA AYUNTAMIENTO DE TABERNO', 'ES304030'],
 'TÍJOLA-DGM' : ['ES304028_1', 'GASTOS DANA AYUNTAMIENTO DE TÍJOLA', 'ES304028', "04"],
 'TORRE PACHECO-DGM': ['ES330010_1', 'GASTOS DANA AYUNTAMIENTO DE TORRE-PACHECO', 'ES330010', "30"],
 'TORRES DE COTILLAS-MAC': ['ES330008_1', 'GASTOS DANA AYUNTAMIENTO DE TORRES DE COTILLAS, LAS', 'ES330008', "30"],
 'TOTANA-JOG': ['ES330032_1', 'GASTOS DANA AYUNTAMIENTO DE TOTANA', 'ES330032', "30"],
 
 'ULEA-JOG': ['ES330029_1', 'GASTOS DANA AYUNTAMIENTO DE ULEA', 'ES330029', "30"],
 'UNIÓN-DGM': ['ES330027_1', 'GASTOS DANA AYUNTAMIENTO DE UNIÓN, LA', 'ES330027', "30"],
 'URRACAL-DGM': ['ES304031_1', 'GASTOS DANA AYUNTAMIENTO DE URRÁCAL', 'ES304031', "04"],
 'VALLADA-JOG': ['ES312038_1', 'GASTOS DANA AYUNTAMIENTO DE VALLADA', 'ES312038', "46"],
 #'VILLANUEVA DEL RIO SEGURA-DGM': ['ES330018_1', 'GASTOS DANA AYUNTAMIENTO DE VILLANUEVA DEL RÍO SEGURA', 'ES330018', "30"],
 'VIVEROS-JOG': ['ES302030_1', 'GASTOS DANA AYUNTAMIENTO DE VIVEROS', 'ES302030', "02"],
 'VÉLEZ MÁLAGA-DGM': ['ES329014_1', 'GASTOS DANA AYUNTAMIENTO DE VÉLEZ-MÁLAGA', 'ES329014', "29"],
 'ZURGENA-JOG': ['ES304023_1', 'GASTOS DANA AYUNTAMIENTO DE ZURGENA', 'ES304023', "04"],
    'UNIÓN-DGM':['ES330027_1','GASTOS DANA AYUNTAMIENTO DE UNIÓN, LA','ES330027','30']}

In [4]:
len(ayuntamientos)

89

In [5]:
'''ayuntamientos= {#'GENERALITAT VALENCIANA-EBB': ['ES209001_1', 'GASTOS DANA GENERALITAT VALENCIANA', 'ES209001', "46"],
    'JUNTA ANDALUCÍA-JOG': ['ES204001_1', 'GASTOS DANA JUNTA DE ANDALUCÍA', 'ES204001', "41"],
                'CONF.HIDRO.SEGURA-MAC': ['ES130006_1', 'GASTOS DANA CONFEDERACION HIDROGRAFICA DEL SEGURA, O.A. DEL MITECO', 'ES130006', "30"],
                'DGCMTM y AU-JOG': ['ES117007_1', 'GASTOS DANA DG CARRETERASV (MITMA)', 'ES117007', "30"],
                'MINISTERIO DEFENSA-EBB': ['ES114007_1', 'GASTOS DANA MINISTERIO DE DEFENSA', 'ES114007', "28"],
    'CONF.HIDRO.JÚCAR-EBB': ['ES130001_1', 'GASTOS DANA  CONFEDERACION HIDROGRAFICA DEL JÚCAR, O.A. DEL MITECO', 'ES130001', "46"],
                'NIJAR-MAC': ['ES304007_1', 'GASTOS DANA AYUNTAMIENTO DE NÍJAR', 'ES304007', "04"],
                'REGIÓN DE MURCIA-DGM': ['ES208001_1', 'GASTOS DANA REGIÓN DE MURCIA', 'ES208001', "30"],
                'ALCÁZARES-MAC': ['ES330025_1','GASTOS DANA AYUNTAMIENTO DE ALCÁZARES, LOS','ES330025', "30"],
                'ALMORADÍ-DGM': ['ES303017_1','GASTOS DANA AYUNTAMIENTO DE ALMORADÍ','ES303017', "46"],
                'ONTINYENT-MAC': ['ES346007_1', 'GASTOS DANA AYUNTAMIENTO DE ONTINYENT', 'ES346007', "46"],
                'ALBAIDA-MAC': ['ES346012_1', 'GASTOS DANA AYUNTAMIENTO DE ALBAIDA', 'ES346012', "46"],
                'BIGASTRO-JOG': ['ES303051_1','GASTOS DANA AYUNTAMIENTO DE BIGASTRO', 'ES303051', "46"],
    'DENIA-MAC': ['ES303026_1', 'GASTOS DANA AYUNTAMIENTO DE DÉNIA', 'ES303026', "03"],            
    'SERÓN-JOG': ['ES304022_1', 'GASTOS DANA AYUNTAMIENTO DE SERÓN', 'ES304022', "04"],
                'DP MÁLAGA-JOG': ['ES329003_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE MÁLAGA', 'ES329003', "29"],
                'ALGORFA-JOG': ['ES303016_1', 'GASTOS DANA AYUNTAMIENTO DE ALGORFA', 'ES303016', "03"],
                'DAYA NUEVA-DGM': ['ES303046_1', 'GASTOS DANA AYUNTAMIENTO DE DAYA NUEVA', 'ES303046', "03"],
                'TÍJOLA-DGM' : ['ES304028_1', 'GASTOS DANA AYUNTAMIENTO DE TÍJOLA', 'ES304028', "04"],
    'PUERTO LUMBRERAS-JOG': ['ES330033_1', 'GASTOS DANA AYUNTAMIENTO DE PUERTO LUMBRERAS', 'ES330033', "30"],            
    'AP CARTAGENA-MAC': ['ES117025_1', 'GASTOS DANA AUTORIDAD PORTUARIA CARTAGENA (MITMA)', 'ES117025', "30"],
                'MOLINICOS-DGM': ['ES302018_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINICOS', 'ES302018', "02"],
                'JACARILA-JOG': ['ES303052_1', 'GASTOS DANA AYUNTAMIENTO DE JACARILLA', 'ES303052', "03"],
                'FUENTE-ÁLAMO-JOG': ['ES302052_1', 'GASTOS DANA AYUNTAMIENTO DE FUENTE-ÁLAMO', 'ES302052', "30"],
                'RIOJA-JOG': ['ES304024_1', 'GASTOS DANA AYUNTAMIENTO DE RIOJA', 'ES304024',"04"],
                'GRANADA-JOG': ['ES318003_1', 'GASTOS DANA AYUNTAMIENTO DE GRANADA', 'ES318003', "18"],
                'ULEA-JOG': ['ES330029_1', 'GASTOS DANA AYUNTAMIENTO DE ULEA', 'ES330029', "30"],
                'AGULLENT-JOG': ['ES303050_1','GASTOS DANA AYUNTAMIENTO DE AGULLENT','ES303050', "46"],
                'LOJA - JOG': ['ES318055_1', 'GASTOS DANA AYUNTAMIENTO DE LOJA', 'ES318055', "18"],
                'BENIEL-JOG': ['ES330013_1', 'GASTOS DANA AYUNTAMIENTO DE BENIEL', 'ES330013', "30"],
                'TORRES DE COTILLAS-MAC': ['ES330008_1', 'GASTOS DANA AYUNTAMIENTO DE TORRES DE COTILLAS, LAS', 'ES330008', "30"],
                'ZURGENA-JOG': ['ES304023_1', 'GASTOS DANA AYUNTAMIENTO DE ZURGENA', 'ES304023', "04"],
                'ALBATERA-JOG': ['ES303044_1','GASTOS DANA AYUNTAMIENTO DE ALBATERA','ES303044', "03"],
                'HUESCAR-DGM': ['ES318042_1', 'GASTOS DANA AYUNTAMIENTO DE HUÉSCAR', 'ES318042', "18"],
                'BLANCA-JOG': ['ES330015_1', 'GASTOS DANA AYUNTAMIENTO DE BLANCA', 'ES330015', "30"],
                'SANTOMERA-JOG': ['ES330014_1', 'GASTOS DANA AYUNTAMIENTO DE SANTOMERA', 'ES330014', "30"],
                'TABERNO - JOG': ['ES304030_1', 'GASTOS DANA AYUNTAMIENTO DE TABERNO', 'ES304030'],
                'OTOS-JOG': ['ES346081_1', 'GASTOS DANA AYUNTAMIENTO DE OTOS', 'ES346081', "46"],
                'TOTANA-JOG': ['ES330032_1', 'GASTOS DANA AYUNTAMIENTO DE TOTANA', 'ES330032', "30"],
                'OJÓS-JOG': ['ES330026_1', 'GASTOS DANA AYUNTAMIENTO DE OJÓS', 'ES330026', "30"],
    'L-OLLERIA-JOG': ['ES346056_1', "GASTOS DANA AYUNTAMIENTO DE OLLERIA, L'", 'ES346056', "46"],            
    'ALCÓNTAR-JOG': ['ES304029_1','GASTOS DANA AYUNTAMIENTO DE ALCÓNTAR', 'ES304029', "04"],
                
}'''

'ayuntamientos= {#\'GENERALITAT VALENCIANA-EBB\': [\'ES209001_1\', \'GASTOS DANA GENERALITAT VALENCIANA\', \'ES209001\', "46"],\n    \'JUNTA ANDALUCÍA-JOG\': [\'ES204001_1\', \'GASTOS DANA JUNTA DE ANDALUCÍA\', \'ES204001\', "41"],\n                \'CONF.HIDRO.SEGURA-MAC\': [\'ES130006_1\', \'GASTOS DANA CONFEDERACION HIDROGRAFICA DEL SEGURA, O.A. DEL MITECO\', \'ES130006\', "30"],\n                \'DGCMTM y AU-JOG\': [\'ES117007_1\', \'GASTOS DANA DG CARRETERASV (MITMA)\', \'ES117007\', "30"],\n                \'MINISTERIO DEFENSA-EBB\': [\'ES114007_1\', \'GASTOS DANA MINISTERIO DE DEFENSA\', \'ES114007\', "28"],\n    \'CONF.HIDRO.JÚCAR-EBB\': [\'ES130001_1\', \'GASTOS DANA  CONFEDERACION HIDROGRAFICA DEL JÚCAR, O.A. DEL MITECO\', \'ES130001\', "46"],\n                \'NIJAR-MAC\': [\'ES304007_1\', \'GASTOS DANA AYUNTAMIENTO DE NÍJAR\', \'ES304007\', "04"],\n                \'REGIÓN DE MURCIA-DGM\': [\'ES208001_1\', \'GASTOS DANA REGIÓN DE MURCIA\', \'ES208001\', "30"],\n          

In [6]:
'''ayuntamientos= {'NIJAR-MAC': ['ES304007_1', 'GASTOS DANA AYUNTAMIENTO DE NÍJAR', 'ES304007', "04"],
                'ALGORFA-JOG': ['ES303016_1', 'GASTOS DANA AYUNTAMIENTO DE ALGORFA', 'ES303016', "03"],
                'TÍJOLA-DGM' : ['ES304028_1', 'GASTOS DANA AYUNTAMIENTO DE TÍJOLA', 'ES304028', "04"],
                'HUESCAR-DGM': ['ES318042_1', 'GASTOS DANA AYUNTAMIENTO DE HUÉSCAR', 'ES318042', "18"],
                'RIOJA-JOG': ['ES304024_1', 'GASTOS DANA AYUNTAMIENTO DE RIOJA', 'ES304024',"04"],
                'CONF.HIDRO.SEGURA-MAC': ['ES130006_1', 'GASTOS DANA CONFEDERACION HIDROGRAFICA DEL SEGURA, O.A. DEL MITECO', 'ES130006', "30"],
               'ALCÁZARES-MAC': ['ES330025_1','GASTOS DANA AYUNTAMIENTO DE ALCÁZARES, LOS','ES330025', "30"],
               'BIGASTRO-JOG': ['ES303051_1','GASTOS DANA AYUNTAMIENTO DE BIGASTRO', 'ES303051', "46"],
               'DAYA NUEVA-DGM': ['ES303046_1', 'GASTOS DANA AYUNTAMIENTO DE DAYA NUEVA', 'ES303046', "03"],
               'FUENTE-ÁLAMO-JOG': ['ES302052_1', 'GASTOS DANA AYUNTAMIENTO DE FUENTE-ÁLAMO', 'ES302052', "30"],
               'ALBATERA-JOG': ['ES303044_1','GASTOS DANA AYUNTAMIENTO DE ALBATERA','ES303044', "03"],
               }
               '''

'ayuntamientos= {\'NIJAR-MAC\': [\'ES304007_1\', \'GASTOS DANA AYUNTAMIENTO DE NÍJAR\', \'ES304007\', "04"],\n                \'ALGORFA-JOG\': [\'ES303016_1\', \'GASTOS DANA AYUNTAMIENTO DE ALGORFA\', \'ES303016\', "03"],\n                \'TÍJOLA-DGM\' : [\'ES304028_1\', \'GASTOS DANA AYUNTAMIENTO DE TÍJOLA\', \'ES304028\', "04"],\n                \'HUESCAR-DGM\': [\'ES318042_1\', \'GASTOS DANA AYUNTAMIENTO DE HUÉSCAR\', \'ES318042\', "18"],\n                \'RIOJA-JOG\': [\'ES304024_1\', \'GASTOS DANA AYUNTAMIENTO DE RIOJA\', \'ES304024\',"04"],\n                \'CONF.HIDRO.SEGURA-MAC\': [\'ES130006_1\', \'GASTOS DANA CONFEDERACION HIDROGRAFICA DEL SEGURA, O.A. DEL MITECO\', \'ES130006\', "30"],\n               \'ALCÁZARES-MAC\': [\'ES330025_1\',\'GASTOS DANA AYUNTAMIENTO DE ALCÁZARES, LOS\',\'ES330025\', "30"],\n               \'BIGASTRO-JOG\': [\'ES303051_1\',\'GASTOS DANA AYUNTAMIENTO DE BIGASTRO\', \'ES303051\', "46"],\n               \'DAYA NUEVA-DGM\': [\'ES303046_1\', \'GAST

In [7]:
'''ayuntamientos= {'RENFE-MAC': ['ES117024_1', 'GASTOS DANA RENFE (MITMA)', 'ES117024', "28"],
                'REGIÓN DE MURCIA-DGM': ['ES208001_1', 'GASTOS DANA REGIÓN DE MURCIA', 'ES208001', "30"],
                'MINISTERIO DEFENSA-EBB': ['ES114007_1', 'GASTOS DANA MINISTERIO DE DEFENSA', 'ES114007', "28"],
                'JUNTA ANDALUCÍA-JOG': ['ES204001_1', 'GASTOS DANA JUNTA DE ANDALUCÍA', 'ES204001', "41"],
                'DP MÁLAGA-JOG': ['ES329003_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE MÁLAGA', 'ES329003', "29"],
                'CONF.HIDRO.JÚCAR-EBB': ['ES130001_1', 'GASTOS DANA  CONFEDERACION HIDROGRAFICA DEL JÚCAR, O.A. DEL MITECO', 'ES130001', "46"],
                'ZURGENA-JOG': ['ES304023_1', 'GASTOS DANA AYUNTAMIENTO DE ZURGENA', 'ES304023', "04"],
                'VIVEROS-JOG': ['ES302030_1', 'GASTOS DANA AYUNTAMIENTO DE VIVEROS', 'ES302030', "02"],
                'VALLADA-JOG': ['ES312038_1', 'GASTOS DANA AYUNTAMIENTO DE VALLADA', 'ES312038',"46"],
                'ULEA-JOG': ['ES330029_1', 'GASTOS DANA AYUNTAMIENTO DE ULEA', 'ES330029', "30"],
                'TOTANA-JOG': ['ES330032_1', 'GASTOS DANA AYUNTAMIENTO DE TOTANA', 'ES330032', "30"],
                'TORRES DE COTILLAS-MAC': ['ES330008_1', 'GASTOS DANA AYUNTAMIENTO DE TORRES DE COTILLAS, LAS', 'ES330008', "30"],
                'TABERNO - JOG': ['ES304030_1', 'GASTOS DANA AYUNTAMIENTO DE TABERNO', 'ES304030'],
                'SERÓN-JOG': ['ES304022_1', 'GASTOS DANA AYUNTAMIENTO DE SERÓN', 'ES304022', "04"],
                'SANTOMERA-JOG': ['ES330014_1', 'GASTOS DANA AYUNTAMIENTO DE SANTOMERA', 'ES330014', "30"],
                'SAN JAVIER-JOG': ['ES330001_1', 'GASTOS DANA AYUNTAMIENTO DE SAN JAVIER', 'ES330001', "30"],
                'SAN FULGENCIO-MAC': ['ES302057_1', 'GASTOS DANA AYUNTAMIENTO DE SAN FULGENCIO', 'ES302057', "03"],
                'PURCHENA-JOG': ['ES304019_1', 'GASTOS DANA AYUNTAMIENTO DE PURCHENA', 'ES304019', "04"],
                'PULPÍ-JOG': ['ES304016_1', 'GASTOS DANA AYUNTAMIENTO DE PULPÍ', 'ES304016', "04"],
                'PUERTO LUMBRERAS-JOG': ['ES330033_1', 'GASTOS DANA AYUNTAMIENTO DE PUERTO LUMBRERAS', 'ES330033', "30"],
                'PINOS GENIL-MAC': ['ES318057_1', 'GASTOS DANA AYUNTAMIENTO DE PINOS GENIL', 'ES318057', "18"],
                'OTOS-JOG': ['ES346081_1', 'GASTOS DANA AYUNTAMIENTO DE OTOS', 'ES346081', "46"],
                'ORIHUELA-MAC': ['ES303003_1', 'GASTOS DANA AYUNTAMIENTO DE ORIHUELA', 'ES303003', "03"],
                'ONTINYENT-MAC': ['ES346007_1', 'GASTOS DANA AYUNTAMIENTO DE ONTINYENT', 'ES346007', "46"],
                'OLULA DEL RÍO-JOG': ['ES304027_1', 'GASTOS DANA AYUNTAMIENTO DE OLULA DEL RÍO', 'ES304027', "04"],
                'L-OLLERIA-JOG': ['ES346056_1', "GASTOS DANA AYUNTAMIENTO DE OLLERIA, L'", 'ES346056', "46"],
                'OJÓS-JOG': ['ES330026_1', 'GASTOS DANA AYUNTAMIENTO DE OJÓS', 'ES330026', "30"],
                'NERPIO-JOG': ['ES302023_1', 'GASTOS DANA AYUNTAMIENTO DE NERPIO', 'ES302023', "02"],
                'MOLINICOS-DGM': ['ES302018_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINICOS', 'ES302018', "02"],
                'MOLINA DEL SEGURA-MAC': ['ES330009_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINA DE SEGURA', 'ES330009', "30"],
                'LORQUÍ-JOG': ['ES330017_1', 'GASTOS DANA AYUNTAMIENTO DE LORQUÍ', 'ES330017', "30"],
                'LOJA - JOG': ['ES318055_1', 'GASTOS DANA AYUNTAMIENTO DE LOJA', 'ES318055', "18"],
                'JACARILA-JOG': ['ES303052_1', 'GASTOS DANA AYUNTAMIENTO DE JACARILLA', 'ES303052', "03"],
                'GRANADA-JOG': ['ES318003_1', 'GASTOS DANA AYUNTAMIENTO DE GRANADA', 'ES318003', "18"],
                'GALLARDOS-JOG': ['ES304026_1', 'GASTOS DANA AYUNTAMIENTO DE GALLARDOS, LOS', 'ES304026', "04"],
                'DENIA-MAC': ['ES303026_1', 'GASTOS DANA AYUNTAMIENTO DE DÉNIA', 'ES303026', "03"],
                'CREVILLENT-MAC': ['ES303047_1', 'GASTOS DANA AYUNTAMIENTO DE CREVILLENT', 'ES303047', "03"],
                'CIEZA-JOG': ['ES330002_1', 'GASTOS DANA AYUNTAMIENTO DE CIEZA', 'ES330002', "30"],
                'BLANCA-JOG': ['ES330015_1', 'GASTOS DANA AYUNTAMIENTO DE BLANCA', 'ES330015', "30"],
                'BENIEL-JOG': ['ES330013_1', 'GASTOS DANA AYUNTAMIENTO DE BENIEL', 'ES330013', "30"],
                'BAZA-MAC': ['ES318001_1', 'GASTOS DANA AYUNTAMIENTO DE BAZA', 'ES318001', "18"],
                'ARCHENA-MAC': ['ES330005_1', 'GASTOS DANA AYUNTAMIENTO DE ARCHENA', 'ES330005', "30"],
                'ARBOLEAS-MAC': ['ES304025_1','GASTOS DANA AYUNTAMIENTO DE ARBOLEAS','ES304025', "04"],
                'ALMORADÍ-DGM': ['ES303017_1','GASTOS DANA AYUNTAMIENTO DE ALMORADÍ','ES303017', "46"],
                'ALHAURÍN DE LA TORRE-MAC': ['ES329005_1','GASTOS DANA AYUNTAMIENTO DE ALHAURÍN DE LA TORRE','ES329005', "29"],
    
                'ALCÓNTAR-JOG': ['ES304029_1','GASTOS DANA AYUNTAMIENTO DE ALCÓNTAR', 'ES304029', "04"],
                'ALBAIDA-MAC': ['ES346012_1', 'GASTOS DANA AYUNTAMIENTO DE ALBAIDA', 'ES346012', "46"],
                'AGULLENT-JOG': ['ES303050_1','GASTOS DANA AYUNTAMIENTO DE AGULLENT','ES303050', "46"],
                'ABANILLA-JOG': ['ES330024_1', 'GASTOS DANA AYUNTAMIENTO DE ABANILLA', 'ES330024', "30"],
                'AP CARTAGENA-MAC': ['ES117025_1', 'GASTOS DANA AUTORIDAD PORTUARIA CARTAGENA (MITMA)', 'ES117025', "30"],
               }'''

'ayuntamientos= {\'RENFE-MAC\': [\'ES117024_1\', \'GASTOS DANA RENFE (MITMA)\', \'ES117024\', "28"],\n                \'REGIÓN DE MURCIA-DGM\': [\'ES208001_1\', \'GASTOS DANA REGIÓN DE MURCIA\', \'ES208001\', "30"],\n                \'MINISTERIO DEFENSA-EBB\': [\'ES114007_1\', \'GASTOS DANA MINISTERIO DE DEFENSA\', \'ES114007\', "28"],\n                \'JUNTA ANDALUCÍA-JOG\': [\'ES204001_1\', \'GASTOS DANA JUNTA DE ANDALUCÍA\', \'ES204001\', "41"],\n                \'DP MÁLAGA-JOG\': [\'ES329003_1\', \'GASTOS DANA DIPUTACIÓN PROVINCIAL DE MÁLAGA\', \'ES329003\', "29"],\n                \'CONF.HIDRO.JÚCAR-EBB\': [\'ES130001_1\', \'GASTOS DANA  CONFEDERACION HIDROGRAFICA DEL JÚCAR, O.A. DEL MITECO\', \'ES130001\', "46"],\n                \'ZURGENA-JOG\': [\'ES304023_1\', \'GASTOS DANA AYUNTAMIENTO DE ZURGENA\', \'ES304023\', "04"],\n                \'VIVEROS-JOG\': [\'ES302030_1\', \'GASTOS DANA AYUNTAMIENTO DE VIVEROS\', \'ES302030\', "02"],\n                \'VALLADA-JOG\': [\'ES31203

In [8]:
ayuntamientos = {'ABANILLA-JOG': ['ES330024_1', 'GASTOS DANA AYUNTAMIENTO DE ABANILLA', 'ES330024', "30"],
 'ABARÁN-EBB': ['ES330031_1', 'GASTOS DANA AYUNTAMIENTO DE ABARÁN', 'ES330031', "30"],
 'AENA-MAC-EBB': ['ES117023_1', 'GASTOS DANA AENA (MITMA)', 'ES117023', "28"],
 'AGULLENT-JOG': ['ES303050_1','GASTOS DANA AYUNTAMIENTO DE AGULLENT','ES303050', "46"],
 'ALBAIDA-MAC': ['ES346012_1', 'GASTOS DANA AYUNTAMIENTO DE ALBAIDA', 'ES346012', "46"],
 'ALBATERA-JOG': ['ES303044_1','GASTOS DANA AYUNTAMIENTO DE ALBATERA','ES303044', "03"],
 'ALCÁZARES-MAC': ['ES330025_1','GASTOS DANA AYUNTAMIENTO DE ALCÁZARES, LOS','ES330025', "30"],
 'ALCÓNTAR-JOG': ['ES304029_1','GASTOS DANA AYUNTAMIENTO DE ALCÓNTAR', 'ES304029', "04"],
 'ALGORFA-JOG': ['ES303016_1', 'GASTOS DANA AYUNTAMIENTO DE ALGORFA', 'ES303016', "03"],
 'ALHAURÍN DE LA TORRE-MAC': ['ES329005_1','GASTOS DANA AYUNTAMIENTO DE ALHAURÍN DE LA TORRE','ES329005', "29"],
 'ALMANSA-EBB': ['ES302002_1', 'GASTOS DANA AYUNTAMIENTO DE ALMANSA', 'ES302002', "02"],
 'ALMERÍA-DGM': ['ES304004_1', 'GASTOS DANA AYUNTAMIENTO DE ALMERÍA', 'ES304004', "04"],
 'ALMORADÍ-DGM': ['ES303017_1','GASTOS DANA AYUNTAMIENTO DE ALMORADÍ','ES303017', "46"],
 "ALQUERIA DE AZNAR-EBB": ['ES303018_1', "GASTOS DANA AYUNTAMIENTO DE ALQUERIA D'ASNAR, L'", 'ES303018', "46"],
 'AP CARTAGENA-MAC': ['ES117025_1', 'GASTOS DANA AUTORIDAD PORTUARIA CARTAGENA (MITMA)', 'ES117025', "30"],
 'ARBOLEAS-MAC': ['ES304025_1','GASTOS DANA AYUNTAMIENTO DE ARBOLEAS','ES304025', "04"],
 'ARCHENA-MAC': ['ES330005_1', 'GASTOS DANA AYUNTAMIENTO DE ARCHENA', 'ES330005', "30"],
 'ARCHIDONA-EBB': ['ES329023_1','GASTOS DANA AYUNTAMIENTO DE ARCHIDONA','ES329023', "29"],
 'BAZA-MAC': ['ES318001_1', 'GASTOS DANA AYUNTAMIENTO DE BAZA', 'ES318001', "18"],
 'BENIEL-JOG': ['ES330013_1', 'GASTOS DANA AYUNTAMIENTO DE BENIEL', 'ES330013', "30"],
 'BIGASTRO-JOG': ['ES303051_1','GASTOS DANA AYUNTAMIENTO DE BIGASTRO', 'ES303051', "46"],
 'BLANCA-JOG': ['ES330015_1', 'GASTOS DANA AYUNTAMIENTO DE BLANCA', 'ES330015', "30"],
 'CAMPOS DEL RÍO - DGM': ['ES330030_1', 'GASTOS DANA AYUNTAMIENTO DE CAMPOS DEL RÍO', 'ES330030', "30"],
 'CASTILLA LA MANCHA- DGM': ['ES211001_1','GASTOS DANA COMUNIDAD AUTÓNOMA DE CASTILLA-LA MANCHA','ES211001', "45"],
 'CASTRIL-DGM': ['ES318054_1', 'GASTOS DANA AYUNTAMIENTO DE CASTRIL', 'ES318054', "18"],
 'CHAUCHINA-JOG': ['ES318053_1', 'GASTOS DANA AYUNTAMIENTO DE CHAUCHINA', 'ES318053', "18"],
 'CIEZA-JOG': ['ES330002_1', 'GASTOS DANA AYUNTAMIENTO DE CIEZA', 'ES330002', "30"],
 #'COCENTAINA-EBB': ['ES303025_1', 'GASTOS DANA AYUNTAMIENTO DE COCENTAINA', 'ES303025', "03"],
 'CONF.HIDRO.JÚCAR-EBB': ['ES130001_1', 'GASTOS DANA  CONFEDERACION HIDROGRAFICA DEL JÚCAR, O.A. DEL MITECO', 'ES130001', "46"],
 'CONF.HIDRO.SEGURA-MAC': ['ES130006_1', 'GASTOS DANA CONFEDERACION HIDROGRAFICA DEL SEGURA, O.A. DEL MITECO', 'ES130006', "30"],
 'COÍN-JOG': ['ES329029_1', 'GASTOS DANA AYUNTAMIENTO DE COÍN', 'ES329029', "29"],
 'CREVILLENT-MAC': ['ES303047_1', 'GASTOS DANA AYUNTAMIENTO DE CREVILLENT', 'ES303047', "03"],
 'DAYA NUEVA-DGM': ['ES303046_1', 'GASTOS DANA AYUNTAMIENTO DE DAYA NUEVA', 'ES303046', "03"],
 'DENIA-MAC': ['ES303026_1', 'GASTOS DANA AYUNTAMIENTO DE DÉNIA', 'ES303026', "03"],
 'DGCMTM y AU-JOG': ['ES117007_1', 'GASTOS DANA DG CARRETERASV (MITMA)', 'ES117007', "30"],
 'DP ALMERÍA- DGM': ['ES304006_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE ALMERÍA', 'ES304006', "04"],
 'DP GRANADA-JOG': ['ES318007_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE GRANADA', 'ES318007', "18"],
 'DP MÁLAGA-JOG': ['ES329003_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE MÁLAGA', 'ES329003', "29"],
 'FUENTE-ÁLAMO-JOG': ['ES302052_1', 'GASTOS DANA AYUNTAMIENTO DE FUENTE-ÁLAMO', 'ES302052', "30"],
 'GALLARDOS-JOG': ['ES304026_1', 'GASTOS DANA AYUNTAMIENTO DE GALLARDOS, LOS', 'ES304026', "04"],
 'GENERALITAT VALENCIANA-EBB': ['ES209001_1', 'GASTOS DANA GENERALITAT VALENCIANA', 'ES209001', "46"],
 'GRANADA-JOG': ['ES318003_1', 'GASTOS DANA AYUNTAMIENTO DE GRANADA', 'ES318003', "18"],
 'HUESCAR-DGM': ['ES318042_1', 'GASTOS DANA AYUNTAMIENTO DE HUÉSCAR', 'ES318042', "18"],
 'JACARILA-JOG': ['ES303052_1', 'GASTOS DANA AYUNTAMIENTO DE JACARILLA', 'ES303052', "03"],
 'JUNTA ANDALUCÍA-JOG': ['ES204001_1', 'GASTOS DANA JUNTA DE ANDALUCÍA', 'ES204001', "41"],
 'L-OLLERIA-JOG': ['ES346056_1', "GASTOS DANA AYUNTAMIENTO DE OLLERIA, L'", 'ES346056', "46"],
 'LOJA - JOG': ['ES318055_1', 'GASTOS DANA AYUNTAMIENTO DE LOJA', 'ES318055',"18"],
 'LORCHA-DGM': ['ES303054_1', "GASTOS DANA AYUNTAMIENTO DE LORCHA/ORXA, L'", 'ES303054', "03"],
 'LORQUÍ-JOG': ['ES330017_1', 'GASTOS DANA AYUNTAMIENTO DE LORQUÍ', 'ES330017', "30"],
 'MINISTERIO DEFENSA-EBB': ['ES114007_1', 'GASTOS DANA MINISTERIO DE DEFENSA', 'ES114007', "28"],
 'MOLINA DEL SEGURA-MAC': ['ES330009_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINA DE SEGURA', 'ES330009', "30"],
 'MOLINICOS-DGM': ['ES302018_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINICOS', 'ES302018', "02"],
 'MONACHIL- DGM': ['ES318026_1', 'GASTOS DANA AYUNTAMIENTO DE MONACHIL', 'ES318026', "18"],
 'NERPIO-JOG': ['ES302023_1', 'GASTOS DANA AYUNTAMIENTO DE NERPIO', 'ES302023', "02"],
 'NIJAR-MAC': ['ES304007_1', 'GASTOS DANA AYUNTAMIENTO DE NÍJAR', 'ES304007', "04"],
 'NOVELDA-EBB': ['ES303045_1', 'GASTOS DANA AYUNTAMIENTO DE NOVELDA', 'ES303045', "03"],
 'OJÓS-JOG': ['ES330026_1', 'GASTOS DANA AYUNTAMIENTO DE OJÓS', 'ES330026', "30"],
 'OLULA DEL RÍO-JOG': ['ES304027_1', 'GASTOS DANA AYUNTAMIENTO DE OLULA DEL RÍO', 'ES304027', "04"],
 'ONTINYENT-MAC': ['ES346007_1', 'GASTOS DANA AYUNTAMIENTO DE ONTINYENT', 'ES346007', "46"],
 'ORIHUELA-MAC': ['ES303003_1', 'GASTOS DANA AYUNTAMIENTO DE ORIHUELA', 'ES303003', "03"],
 'OTOS-JOG': ['ES346081_1', 'GASTOS DANA AYUNTAMIENTO DE OTOS', 'ES346081', "46"],
 'PARCENT-EBB': ['ES303049_1', 'GASTOS DANA AYUNTAMIENTO DE PARCENT', 'ES303049', "03"],
 'PILAR DE LA HORADADA EBB': ['ES303041_1', 'GASTOS DANA AYUNTAMIENTO DE PILAR DE LA HORADADA', 'ES303041', "03"],
 'PINOS GENIL-MAC': ['ES318057_1', 'GASTOS DANA AYUNTAMIENTO DE PINOS GENIL', 'ES318057', "18"],
 'PUEBLA DE DON FADRIQUE-EBB': ['ES318043_1', 'GASTOS DANA AYUNTAMIENTO DE PUEBLA DE DON FADRIQUE', 'ES318043', "18"],
 'PUERTO LUMBRERAS-JOG': ['ES330033_1', 'GASTOS DANA AYUNTAMIENTO DE PUERTO LUMBRERAS', 'ES330033', "30"],
 'PULPÍ-JOG': ['ES304016_1', 'GASTOS DANA AYUNTAMIENTO DE PULPÍ', 'ES304016', "04"],
 'PURCHENA-JOG': ['ES304019_1', 'GASTOS DANA AYUNTAMIENTO DE PURCHENA', 'ES304019', "04"],
 'REDOVÁN-DGM': ['ES303048_1', 'GASTOS DANA AYUNTAMIENTO DE REDOVÁN', 'ES303048', "03"],
 'REGIÓN DE MURCIA-DGM': ['ES208001_1', 'GASTOS DANA REGIÓN DE MURCIA', 'ES208001', "30"],
 'RENFE-MAC': ['ES117024_1', 'GASTOS DANA RENFE (MITMA)', 'ES117024', "28"],
 'RINCÓN DE LA VICTORIA-DGM': ['ES329011_1', 'GASTOS DANA AYUNTAMIENTO DE RINCÓN DE LA VICTORIA', 'ES329011', "29"],
 'RIOJA-JOG': ['ES304024_1', 'GASTOS DANA AYUNTAMIENTO DE RIOJA', 'ES304024',"04"],
 'SAN FULGENCIO-MAC': ['ES302057_1', 'GASTOS DANA AYUNTAMIENTO DE SAN FULGENCIO', 'ES302057', "03"],
 'SAN JAVIER-JOG': ['ES330001_1', 'GASTOS DANA AYUNTAMIENTO DE SAN JAVIER', 'ES330001', "30"],
 'SANTOMERA-JOG': ['ES330014_1', 'GASTOS DANA AYUNTAMIENTO DE SANTOMERA', 'ES330014', "30"],
 'SERÓN-JOG': ['ES304022_1', 'GASTOS DANA AYUNTAMIENTO DE SERÓN', 'ES304022', "04"],
 'TABERNO - JOG': ['ES304030_1', 'GASTOS DANA AYUNTAMIENTO DE TABERNO', 'ES304030'],
 'TÍJOLA-DGM' : ['ES304028_1', 'GASTOS DANA AYUNTAMIENTO DE TÍJOLA', 'ES304028', "04"],
 'TORRE PACHECO-DGM': ['ES330010_1', 'GASTOS DANA AYUNTAMIENTO DE TORRE-PACHECO', 'ES330010', "30"],
 'TORRES DE COTILLAS-MAC': ['ES330008_1', 'GASTOS DANA AYUNTAMIENTO DE TORRES DE COTILLAS, LAS', 'ES330008', "30"],
 'TOTANA-JOG': ['ES330032_1', 'GASTOS DANA AYUNTAMIENTO DE TOTANA', 'ES330032', "30"],
 'ULEA-JOG': ['ES330029_1', 'GASTOS DANA AYUNTAMIENTO DE ULEA', 'ES330029', "30"],
 'UNIÓN-DGM': ['ES330027_1', 'GASTOS DANA AYUNTAMIENTO DE UNIÓN, LA', 'ES330027', "30"],
 'URRACAL-DGM': ['ES304031_1', 'GASTOS DANA AYUNTAMIENTO DE URRÁCAL', 'ES304031', "04"],
 'VALLADA-JOG': ['ES312038_1', 'GASTOS DANA AYUNTAMIENTO DE VALLADA', 'ES312038', "46"],
 'VIVEROS-JOG': ['ES302030_1', 'GASTOS DANA AYUNTAMIENTO DE VIVEROS', 'ES302030', "02"],
 'VÉLEZ MÁLAGA-DGM': ['ES329014_1', 'GASTOS DANA AYUNTAMIENTO DE VÉLEZ-MÁLAGA', 'ES329014', "29"],
 'ZURGENA-JOG': ['ES304023_1', 'GASTOS DANA AYUNTAMIENTO DE ZURGENA', 'ES304023', "04"],
    'UNIÓN-DGM':['ES330027_1','GASTOS DANA AYUNTAMIENTO DE UNIÓN, LA','ES330027','30']}

In [9]:
# últimos
ayuntamientos = {'ABANILLA-JOG': ['ES330024_1', 'GASTOS DANA AYUNTAMIENTO DE ABANILLA', 'ES330024', "30"],
 'ABARÁN-EBB': ['ES330031_1', 'GASTOS DANA AYUNTAMIENTO DE ABARÁN', 'ES330031', "30"],
 'AENA-MAC-EBB': ['ES117023_1', 'GASTOS DANA AENA (MITMA)', 'ES117023', "28"],
 'AGULLENT-JOG': ['ES303050_1','GASTOS DANA AYUNTAMIENTO DE AGULLENT','ES303050', "46"],
 'ALBAIDA-MAC': ['ES346012_1', 'GASTOS DANA AYUNTAMIENTO DE ALBAIDA', 'ES346012', "46"],
 'ALBATERA-JOG': ['ES303044_1','GASTOS DANA AYUNTAMIENTO DE ALBATERA','ES303044', "03"],
 'ALCÁZARES-MAC': ['ES330025_1','GASTOS DANA AYUNTAMIENTO DE ALCÁZARES, LOS','ES330025', "30"],
 'ALCÓNTAR-JOG': ['ES304029_1','GASTOS DANA AYUNTAMIENTO DE ALCÓNTAR', 'ES304029', "04"],
 'ALGORFA-JOG': ['ES303016_1', 'GASTOS DANA AYUNTAMIENTO DE ALGORFA', 'ES303016', "03"],
 'ALHAURÍN DE LA TORRE-MAC': ['ES329005_1','GASTOS DANA AYUNTAMIENTO DE ALHAURÍN DE LA TORRE','ES329005', "29"],
 'ALMANSA-EBB': ['ES302002_1', 'GASTOS DANA AYUNTAMIENTO DE ALMANSA', 'ES302002', "02"],
 'ALMERÍA-DGM': ['ES304004_1', 'GASTOS DANA AYUNTAMIENTO DE ALMERÍA', 'ES304004', "04"],
 'ALMORADÍ-DGM': ['ES303017_1','GASTOS DANA AYUNTAMIENTO DE ALMORADÍ','ES303017', "46"],
 "ALQUERIA DE AZNAR-EBB": ['ES303018_1', "GASTOS DANA AYUNTAMIENTO DE ALQUERIA D'ASNAR, L'", 'ES303018', "46"],
 'AP CARTAGENA-MAC': ['ES117025_1', 'GASTOS DANA AUTORIDAD PORTUARIA CARTAGENA (MITMA)', 'ES117025', "30"],
 'ARBOLEAS-MAC': ['ES304025_1','GASTOS DANA AYUNTAMIENTO DE ARBOLEAS','ES304025', "04"],
 'ARCHENA-MAC': ['ES330005_1', 'GASTOS DANA AYUNTAMIENTO DE ARCHENA', 'ES330005', "30"],
 'ARCHIDONA-EBB': ['ES329023_1','GASTOS DANA AYUNTAMIENTO DE ARCHIDONA','ES329023', "29"],
 'BAZA-MAC': ['ES318001_1', 'GASTOS DANA AYUNTAMIENTO DE BAZA', 'ES318001', "18"],
 'BENIEL-JOG': ['ES330013_1', 'GASTOS DANA AYUNTAMIENTO DE BENIEL', 'ES330013', "30"],
 'BIGASTRO-JOG': ['ES303051_1','GASTOS DANA AYUNTAMIENTO DE BIGASTRO', 'ES303051', "46"],
 'BLANCA-JOG': ['ES330015_1', 'GASTOS DANA AYUNTAMIENTO DE BLANCA', 'ES330015', "30"],
 'CAMPOS DEL RÍO - DGM': ['ES330030_1', 'GASTOS DANA AYUNTAMIENTO DE CAMPOS DEL RÍO', 'ES330030', "30"],
 'CASTILLA LA MANCHA- DGM': ['ES211001_1','GASTOS DANA COMUNIDAD AUTÓNOMA DE CASTILLA-LA MANCHA','ES211001', "45"],
 'CASTRIL-DGM': ['ES318054_1', 'GASTOS DANA AYUNTAMIENTO DE CASTRIL', 'ES318054', "18"],
 'CHAUCHINA-JOG': ['ES318053_1', 'GASTOS DANA AYUNTAMIENTO DE CHAUCHINA', 'ES318053', "18"],
 'CIEZA-JOG': ['ES330002_1', 'GASTOS DANA AYUNTAMIENTO DE CIEZA', 'ES330002', "30"],
 'COCENTAINA-EBB': ['ES303025_1', 'GASTOS DANA AYUNTAMIENTO DE COCENTAINA', 'ES303025', "03"],
 'CONF.HIDRO.JÚCAR-EBB': ['ES130001_1', 'GASTOS DANA  CONFEDERACION HIDROGRAFICA DEL JÚCAR, O.A. DEL MITECO', 'ES130001', "46"],
 'CONF.HIDRO.SEGURA-MAC': ['ES130006_1', 'GASTOS DANA CONFEDERACION HIDROGRAFICA DEL SEGURA, O.A. DEL MITECO', 'ES130006', "30"],
 'COÍN-JOG': ['ES329029_1', 'GASTOS DANA AYUNTAMIENTO DE COÍN', 'ES329029', "29"],
 'CREVILLENT-MAC': ['ES303047_1', 'GASTOS DANA AYUNTAMIENTO DE CREVILLENT', 'ES303047', "03"],
 'DAYA NUEVA-DGM': ['ES303046_1', 'GASTOS DANA AYUNTAMIENTO DE DAYA NUEVA', 'ES303046', "03"],
 'DENIA-MAC': ['ES303026_1', 'GASTOS DANA AYUNTAMIENTO DE DÉNIA', 'ES303026', "03"],
 'DGCMTM y AU-JOG': ['ES117007_1', 'GASTOS DANA DG CARRETERAS (MITMA)', 'ES117007', "30"],
 'DP ALMERÍA- DGM': ['ES304006_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE ALMERÍA', 'ES304006', "04"],
 'DP GRANADA-JOG': ['ES318007_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE GRANADA', 'ES318007', "18"],
 'DP MÁLAGA-JOG': ['ES329003_1', 'GASTOS DANA DIPUTACIÓN PROVINCIAL DE MÁLAGA', 'ES329003', "29"],
 'FUENTE-ÁLAMO-JOG': ['ES302052_1', 'GASTOS DANA AYUNTAMIENTO DE FUENTE-ÁLAMO', 'ES302052', "30"],
 'GALLARDOS-JOG': ['ES304026_1', 'GASTOS DANA AYUNTAMIENTO DE GALLARDOS, LOS', 'ES304026', "04"],
 'GENERALITAT VALENCIANA-EBB': ['ES209001_1', 'GASTOS DANA GENERALITAT VALENCIANA', 'ES209001', "46"],
 'GRANADA-JOG': ['ES318003_1', 'GASTOS DANA AYUNTAMIENTO DE GRANADA', 'ES318003', "18"],
 'HUESCAR-DGM': ['ES318042_1', 'GASTOS DANA AYUNTAMIENTO DE HUÉSCAR', 'ES318042', "18"],
 'JACARILA-JOG': ['ES303052_1', 'GASTOS DANA AYUNTAMIENTO DE JACARILLA', 'ES303052', "03"],
 'JUNTA ANDALUCÍA-JOG': ['ES204001_1', 'GASTOS DANA JUNTA DE ANDALUCÍA', 'ES204001', "41"],
 'L-OLLERIA-JOG': ['ES346056_1', "GASTOS DANA AYUNTAMIENTO DE OLLERIA, L'", 'ES346056', "46"],
 'LOJA - JOG': ['ES318055_1', 'GASTOS DANA AYUNTAMIENTO DE LOJA', 'ES318055',"18"],
 'LORCHA-DGM': ['ES303054_1', "GASTOS DANA AYUNTAMIENTO DE LORCHA/ORXA, L'", 'ES303054', "03"],
 'LORQUÍ-JOG': ['ES330017_1', 'GASTOS DANA AYUNTAMIENTO DE LORQUÍ', 'ES330017', "30"],
 'MINISTERIO DEFENSA-EBB': ['ES114007_1', 'GASTOS DANA MINISTERIO DE DEFENSA', 'ES114007', "28"],
 'MOLINA DEL SEGURA-MAC': ['ES330009_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINA DE SEGURA', 'ES330009', "30"],
 'MOLINICOS-DGM': ['ES302018_1', 'GASTOS DANA AYUNTAMIENTO DE MOLINICOS', 'ES302018', "02"],
 'MONACHIL- DGM': ['ES318026_1', 'GASTOS DANA AYUNTAMIENTO DE MONACHIL', 'ES318026', "18"],
 'NERPIO-JOG': ['ES302023_1', 'GASTOS DANA AYUNTAMIENTO DE NERPIO', 'ES302023', "02"],
 'NIJAR-MAC': ['ES304007_1', 'GASTOS DANA AYUNTAMIENTO DE NÍJAR', 'ES304007', "04"],
 'NOVELDA-EBB': ['ES303045_1', 'GASTOS DANA AYUNTAMIENTO DE NOVELDA', 'ES303045', "03"],
 'OJÓS-JOG': ['ES330026_1', 'GASTOS DANA AYUNTAMIENTO DE OJÓS', 'ES330026', "30"],
 'OLULA DEL RÍO-JOG': ['ES304027_1', 'GASTOS DANA AYUNTAMIENTO DE OLULA DEL RÍO', 'ES304027', "04"],
 'ONTINYENT-MAC': ['ES346007_1', 'GASTOS DANA AYUNTAMIENTO DE ONTINYENT', 'ES346007', "46"],
 'ORIHUELA-MAC': ['ES303003_1', 'GASTOS DANA AYUNTAMIENTO DE ORIHUELA', 'ES303003', "03"],
 'OTOS-JOG': ['ES346081_1', 'GASTOS DANA AYUNTAMIENTO DE OTOS', 'ES346081', "46"],
 'PARCENT-EBB': ['ES303049_1', 'GASTOS DANA AYUNTAMIENTO DE PARCENT', 'ES303049', "03"],
 'PILAR DE LA HORADADA EBB': ['ES303041_1', 'GASTOS DANA AYUNTAMIENTO DE PILAR DE LA HORADADA', 'ES303041', "03"],
 'PINOS GENIL-MAC': ['ES318057_1', 'GASTOS DANA AYUNTAMIENTO DE PINOS GENIL', 'ES318057', "18"],
 'PUEBLA DE DON FADRIQUE-EBB': ['ES318043_1', 'GASTOS DANA AYUNTAMIENTO DE PUEBLA DE DON FADRIQUE', 'ES318043', "18"],
 'PUERTO LUMBRERAS-JOG': ['ES330033_1', 'GASTOS DANA AYUNTAMIENTO DE PUERTO LUMBRERAS', 'ES330033', "30"],
 'PULPÍ-JOG': ['ES304016_1', 'GASTOS DANA AYUNTAMIENTO DE PULPÍ', 'ES304016', "04"],
 'PURCHENA-JOG': ['ES304019_1', 'GASTOS DANA AYUNTAMIENTO DE PURCHENA', 'ES304019', "04"],
 'REDOVÁN-DGM': ['ES303048_1', 'GASTOS DANA AYUNTAMIENTO DE REDOVÁN', 'ES303048', "03"],
 'REGIÓN DE MURCIA-DGM': ['ES208001_1', 'GASTOS DANA REGIÓN DE MURCIA', 'ES208001', "30"],
 'RENFE-MAC': ['ES117024_1', 'GASTOS DANA RENFE (MITMA)', 'ES117024', "28"],
 'RINCÓN DE LA VICTORIA-DGM': ['ES329011_1', 'GASTOS DANA AYUNTAMIENTO DE RINCÓN DE LA VICTORIA', 'ES329011', "29"],
 'RIOJA-JOG': ['ES304024_1', 'GASTOS DANA AYUNTAMIENTO DE RIOJA', 'ES304024',"04"],
 'SAN FULGENCIO-MAC': ['ES302057_1', 'GASTOS DANA AYUNTAMIENTO DE SAN FULGENCIO', 'ES302057', "03"],
 'SAN JAVIER-JOG': ['ES330001_1', 'GASTOS DANA AYUNTAMIENTO DE SAN JAVIER', 'ES330001', "30"],
 'SANTOMERA-JOG': ['ES330014_1', 'GASTOS DANA AYUNTAMIENTO DE SANTOMERA', 'ES330014', "30"],
 'SERÓN-JOG': ['ES304022_1', 'GASTOS DANA AYUNTAMIENTO DE SERÓN', 'ES304022', "04"],
 'TABERNO - JOG': ['ES304030_1', 'GASTOS DANA AYUNTAMIENTO DE TABERNO', 'ES304030'],
 'TÍJOLA-DGM' : ['ES304028_1', 'GASTOS DANA AYUNTAMIENTO DE TÍJOLA', 'ES304028', "04"],
 'TORRE PACHECO-DGM': ['ES330010_1', 'GASTOS DANA AYUNTAMIENTO DE TORRE-PACHECO', 'ES330010', "30"],
 'TORRES DE COTILLAS-MAC': ['ES330008_1', 'GASTOS DANA AYUNTAMIENTO DE TORRES DE COTILLAS, LAS', 'ES330008', "30"],
 'TOTANA-JOG': ['ES330032_1', 'GASTOS DANA AYUNTAMIENTO DE TOTANA', 'ES330032', "30"],
 'ULEA-JOG': ['ES330029_1', 'GASTOS DANA AYUNTAMIENTO DE ULEA', 'ES330029', "30"],
 'UNIÓN-DGM': ['ES330027_1', 'GASTOS DANA AYUNTAMIENTO DE UNIÓN, LA', 'ES330027', "30"],
 'URRACAL-DGM': ['ES304031_1', 'GASTOS DANA AYUNTAMIENTO DE URRÁCAL', 'ES304031', "04"],
 'VALLADA-JOG': ['ES312038_1', 'GASTOS DANA AYUNTAMIENTO DE VALLADA', 'ES312038', "46"],
 'VIVEROS-JOG': ['ES302030_1', 'GASTOS DANA AYUNTAMIENTO DE VIVEROS', 'ES302030', "02"],
 'VÉLEZ MÁLAGA-DGM': ['ES329014_1', 'GASTOS DANA AYUNTAMIENTO DE VÉLEZ-MÁLAGA', 'ES329014', "29"],
 'ZURGENA-JOG': ['ES304023_1', 'GASTOS DANA AYUNTAMIENTO DE ZURGENA', 'ES304023', "04"],
}

In [10]:
len(ayuntamientos)

89

In [11]:
ayuntamientos={'OLULA DEL RÍO-JOG': ['ES304027_1', 'GASTOS DANA AYUNTAMIENTO DE OLULA DEL RÍO', 'ES304027', "04"],}

In [12]:
fichero_padre = "P:/Regulacion/49 Encargo MinHac 20-22/08 FSUE-DANA/02 Archivos de trabajo/BENEFICIARIOS FSUE DANA"
#fichero_padre = "C:/Users/bllanos/Desktop/Simulacion_dana"

In [13]:
def ayuntamientos_con_el_fichero(aytsdic):
    aytos_sin_fichero = {}
    for k,v in aytsdic.items():
        if "Justificacion_FSUE-DANA.xlsx" not in os.listdir(os.path.join(fichero_padre,k)):
            aytos_sin_fichero.update({k: os.listdir(os.path.join(fichero_padre,k))[4:]})
    return aytos_sin_fichero

In [14]:
sin_fichero = ayuntamientos_con_el_fichero(ayuntamientos)

In [15]:
sin_fichero, len(sin_fichero)

({}, 0)

In [16]:
def getorganismos():
    dicorganismos = {}
    with open (os.path.join(fichero_padre,"data_blp/ORGANISMOS.csv"), encoding="utf-8") as f:
        csv_reader = csv.DictReader(f,delimiter=";")
        for row in csv_reader:
            try:
                dicorganismos[row[" NIF"]].append(row["\ufeffCódigo organismo"])
            except:
                dicorganismos.update({row[" NIF"]:[row["\ufeffCódigo organismo"]]})
    return dicorganismos

In [17]:
NIF_cod = getorganismos()

In [18]:
print(NIF_cod)

{'S2826015F': ['ES101001', 'ES107003'], 'S2826046A': ['ES101002'], 'S2833002E': ['ES101003'], 'S2800223F': ['ES101004', 'ES101005', 'ES101005', 'ES107001', 'ES107002', 'ES115001', 'ES115001', 'ES115002', 'ES115002', 'ES115002', 'ES115003', 'ES115003', 'ES115003', 'ES115007', 'ES115007', 'ES115007', 'ES115007', 'ES115007', 'ES115007', 'ES115010'], 'Q3667002D': ['ES101010', 'ES101010'], 'Q2820009E': ['ES102002', 'ES102003', 'ES120003'], 'Q2829013H': ['ES102004', 'ES110001'], 'A79365821': ['ES103001', 'ES103001', 'ES103001', 'ES103001', 'ES103001', 'ES103001'], 'Q0617001C': ['ES103002', 'ES130004'], 'Q3200109A': ['ES103003'], 'S2800653D': ['ES104001'], 'Q3500385D': ['ES105001', 'ES105001', 'ES404029'], 'Q2820002J': ['ES105002', 'ES105002', 'ES105002', 'ES105030'], 'G85296226': ['ES105003', 'ES105003', 'ES499003'], 'Q0801209H': ['ES105004', 'ES202122', 'ES404024'], 'Q3811001A': ['ES105005', 'ES105037', 'ES105037', 'ES127011', 'ES127011'], 'G62426937': ['ES105007', 'ES105007', 'ES105007', '

In [19]:
def get_df_contratos(filepath):
    df = pd.read_excel(filepath, sheet_name= "1. DATOS BÁSICOS CONTRATACIÓN")
    df.dropna(thresh=5,inplace=True)
    df.columns = df.iloc[0]
    df.drop(df.index[0],inplace=True)
    df.reset_index(inplace=True,drop=True)
    #display(df)
    #print(df.dtypes)
    df["NIF del Organismo Adjudicador:"] = df["NIF del Organismo Adjudicador:"].apply(lambda x : x.replace("-", "").replace(".", "").replace("ES","").replace("(","").replace(")","").replace("\n","").replace(" ","").upper())
    df["Presupuesto de la Licitación (con IVA)"] = pd.Series([round(val,2) for val in df["Presupuesto de la Licitación (con IVA)"]])
    df["Presupuesto de la Licitación (sin IVA)"] = pd.Series([round(val,2) for val in df["Presupuesto de la Licitación (sin IVA)"]])
    df["Importe Adjudicado en la licitación (sin IVA)"] = pd.Series([round(val,2) for val in df["Importe Adjudicado en la licitación (sin IVA)"]])
    df["Importe de ayuda del Fondo de Solidaridad que se solicita (con IVA)**"] = pd.Series([round(val,2) for val in df["Importe de ayuda del Fondo de Solidaridad que se solicita (con IVA)**"]])
    df["Importe de ayuda del Fondo de Solidaridad que se solicita (sin IVA)**"] = pd.Series([round(val,2) for val in df["Importe de ayuda del Fondo de Solidaridad que se solicita (sin IVA)**"]])
    if df.shape[0] != 0:
        df = df[df.elegible != "NO"]
        df = df[df.elegible != "N"]
        df = df[df.elegible != "n"]
        df = df[df.elegible != "no"]
    #df.drop("NaN", axis=1, inplace=True)
    return df

In [20]:
def cod_tipocon_norma(tipo):
    #print(tipo)
    tipo =tipo.split("/")[0]
    #print(tipo)
    if "obra" in tipo.casefold().lower() or "obras" in tipo.casefold().lower():
        return "01"
    elif "servicio" in tipo.casefold().lower() or "servicios" in tipo.casefold().lower():
        return "02"
    elif "suministro" in tipo.casefold().lower() or "suministros" in tipo.casefold().lower():
        return "03"
    elif "gestion de servicios publicos" in tipo.casefold().lower():
        return "04"
    elif "concesion de obra publica" in tipo.casefold().lower():
        return "05"
    elif "concesion entre el sector publico y privado" in tipo.casefold().lower():
        return "06"
    else:
        return "error"

In [21]:
def cod_prod_adj(tipo1, tipo2):
    if tipo1 != "nan":
        tipo = tipo1
        #print(tipo)
        #print(tipo.casefold().lower().find('negociado sin publicidad'))
        if "abierto" in tipo.casefold().lower():
            return "01"
        elif "restringido" in tipo.casefold().lower():
            return "02"
        elif "negociado con publicidad" in tipo.casefold().lower() or tipo.casefold().lower().find('negociado con publicidad') >= 0:
            return "03"
        elif "negociado sin publicidad" in tipo.casefold().lower() or tipo.casefold().lower().find('negociado sin publicidad') >= 0:
            return "04"
        elif "dialogo competititivo" in tipo.casefold().lower():
            return "05"
        elif "concursos de proyecyos" in tipo.casefold().lower():
            return "06"
        elif "contrato menor"in tipo.casefold().lower() or "menor" in tipo.casefold().lower():
            return "08"
        elif "emergencia" in tipo.casefold().lower():
            return "10"
        elif "otro caso" in tipo.casefold().lower() or "encomienda" in tipo.casefold().lower() or tipo.casefold().lower().find('encargo') >= 0 or tipo.casefold().lower().find('encomienda') > 0:
            return "11"
        elif "acuerdo marco" in tipo.casefold().lower():
            return "12"
        else:
            return "error"
    else:
        tipo = tipo2
        if "emergencia" in tipo.casefold().lower():
            return "10"
        else:
            return "error"

In [22]:
def cod_tramitacion(tipo):
    if "ordinaria" in tipo.casefold().lower() or "ordinario" in tipo.casefold().lower():
        return "O"
    elif "urgente" in tipo.casefold().lower() or "urgencia" in tipo.casefold().lower():
        return "U"
    elif "emergencia" in tipo.casefold().lower():
        return "E"
    else:
        return "error"

In [23]:
def codigoscontratosaexcel(ayto,cod_contrato,expediente):
    with open (os.path.join(fichero_padre,"data_blp/correspcontratos.csv"), "a", encoding="latin1") as f:
        f.write(ayto+";"+expediente.replace("\n","")+";"+cod_contrato+"\n")

In [24]:
def tituloselect(texto1, texto2):
    if len(str(texto1)) > len(str(texto2)):
        return str(texto1)
    else:
        return str(texto2)

In [25]:
def getadjudicador(NIF):
    try: 
        if len(NIF_cod[NIF]) == 1:
            return str(NIF_cod[NIF][0])
        else:
            return False
    except KeyError:
        return False

In [26]:
def getadjudicatario(NIF):
    NIF = NIF.replace("ES","").replace(" ","").replace("-","").replace(".","")
    return NIF_cod[NIF][0]

In [27]:
def getduration(time):
    try:
        return str(float(time))
    except ValueError :
        if 'mes' in time.lower() or 'meses' in time.lower():
            return str(float(time.split(" ")[0]))
        else:
            return False

In [28]:
def get_publication(platform):
    if "bdns" in platform.casefold().lower():
        return "BDNS"
    elif "boa" in platform.casefold().lower() or "boletin autonomico" in platform.casefold().lower():
        return "BOA"
    elif "boe" in platform.casefold().lower() or "boletin oficial" in platform.casefold().lower():
        return "BOE"
    elif "bol" in platform.casefold().lower() or "boletin local" in platform.casefold().lower():
        return "BOL"
    elif "doue" in platform.casefold().lower() or "boletin local" in platform.casefold().lower():
        return "DOUE"
    elif "pec" in platform.casefold().lower() or "perfil del contratante" in platform.casefold().lower()  or "perfil contratante" in platform.casefold().lower():
        return "PEC"
    elif "web" in platform.casefold().lower():
        return "WEB"
    elif "PTC" in platform.casefold().lower() or "plataforma contratación" in platform.casefold().lower() or "plataforma de contratación" in platform.casefold().lower() or "Plataforma contratación del Sector Público" in platform.casefold().lower() or "Plataforma de contratación de sector público" in platform.casefold().lower():
        return "DOUE"
    else:
        return "OTR"

In [29]:
%%time
if os.path.exists(os.path.join(fichero_padre,"data_blp/correspcontratos.csv")) == True:
    os.remove(os.path.join(fichero_padre,"data_blp/correspcontratos.csv"))
empty = []
facturas = {}
organismos ={}
error_null = {}
error_textos = 0
root_contratos = ET.Element("contratos")
for k,v in ayuntamientos.items():
    if k not in sin_fichero:
        print(k)
        df=get_df_contratos(os.path.join(fichero_padre,k,"Justificacion_FSUE-DANA.xlsx"))
        if df.shape[0] == 0:
            empty.append(k)
            print(f"{k} no tiene contratos")
        else:
            #contratos = ET.Element("contratos")
            num_contratos = 1
            display(df)
            for index, row in df.iterrows():
                contrato = ET.SubElement(root_contratos,"contrato")
                cod_contrato = ET.SubElement(contrato,"cod_contrato_benef")
                cod_contrato.text = v[2]+"_"+str(num_contratos)
                codigoscontratosaexcel(k,v[2]+"_"+str(num_contratos),str(row["Código Expte de Contratación/Encargo"]))
                if len(row["Denominación del Contrato/Encargo"]) > 200:
                    error_textos += 1
                nom_contrato = ET.SubElement(contrato,"nombre")
                nom_contrato.text = str(row["Código Expte de Contratación/Encargo"])[0:200]
                titulo_largo = ET.SubElement(contrato,"titulo_largo")
                #print(row["Denominación del Contrato/Encargo"])
                titulo_largo.text = tituloselect(str(row["Denominación del Contrato/Encargo"]),str(row["Objeto Contrato/Encargo"]))
                programas = ET.SubElement(contrato,"programas_operativos")
                programa = ET.SubElement(programas,"programa_operativo")
                codigo = ET.SubElement(programa,"codigo_poperativo")
                codigo.text = "SO-DANA"
                normativa = ET.SubElement(contrato,"cod_normativa")
                normativa.text = "L9_2017"
                cod_tipo_norma = ET.SubElement(contrato,"cod_tipocon_norma")
                cod_tipo_norma.text = cod_tipocon_norma(str(row["Tipo Contrato (Servicios, suministros, etc.)/Encargo"]))
                objeto = ET.SubElement(contrato,"objeto_contrato")
                objeto.text = str(row["Objeto Contrato/Encargo"])
                #if pd.isnull(row["Tipo Procedimiento (Abierto, Negociado con/sin publicidad, Acuerdo Marco, Contrato Menor)"]) == False:
                cod_proc_adj = ET.SubElement(contrato,"cod_proc_adj")
                cod_proc_adj.text = cod_prod_adj(str(row["Tipo Procedimiento (Abierto, Negociado con/sin publicidad, Acuerdo Marco, Contrato Menor)"]),str(row["Tipo Tramitación (Ordinaria, Urgencia, Emergencia)"]))
                tramitacion = ET.SubElement(contrato,"tramitacion")
                tramitacion.text = cod_tramitacion(str(row["Tipo Tramitación (Ordinaria, Urgencia, Emergencia)"]))
                if pd.isnull(row["Fecha (dd/mm/aaaa)"]) == False and pd.isnull(row["Medio (BOE, DOUE, BDNS, Perfil Contratante, etc.)"]) == False:
                    fechas_lic = ET.SubElement(contrato,"fechas_publicacion_licitacion")
                    fecha_lic = ET.SubElement(fechas_lic,"fecha_publicacion_licitacion")
                    medio = ET.SubElement(fecha_lic,"cod_medio_pub")
                    medio.text = get_publication(str(row["Medio (BOE, DOUE, BDNS, Perfil Contratante, etc.)"]))
                    f_pub = ET.SubElement(fecha_lic,"f_publicacion")
                    f_pub.text = str(row["Fecha (dd/mm/aaaa)"].strftime("%d/%m/%Y"))
                    desc = ET.SubElement(fecha_lic,"descripcion")
                    desc.text = " "
                imp_licitacion = ET.SubElement(contrato,"imp_licitacion")
                if pd.isnull(row["Presupuesto de la Licitación (con IVA)"]) == False:
                    imp_licitacion.text = str(row["Presupuesto de la Licitación (con IVA)"])
                else:
                    imp_licitacion.text = str(row["Importe Adjudicado en la licitación (con IVA)"])
                if pd.isnull(row["Duración contrato/Encargo (meses)"])== False:
                    duracion = ET.SubElement(contrato,"duracion_contrato")
                    duracion.text = getduration(str(row["Duración contrato/Encargo (meses)"])) if getduration(str(row["Duración contrato/Encargo (meses)"]))!= False else "0"
                valor = ET.SubElement(contrato,"valor_estimado")
                if pd.isnull(row["Presupuesto de la Licitación (sin IVA)"]) == False:
                    valor.text = str(row["Importe Adjudicado en la licitación (sin IVA)"])
                else:
                    valor.text = str(row["Importe Adjudicado en la licitación (con IVA)"])
                imp_adjudicado = ET.SubElement(contrato,"importe_adjudicado")
                imp_adjudicado.text = str(row["Importe Adjudicado en la licitación (sin IVA)"])
                pres_vig = ET.SubElement(contrato,"presupuesto_vigente")
                pres_vig.text = str(row["Importe Adjudicado en la licitación (con IVA)"])
                cod_org_adjudica = ET.SubElement(contrato,"cod_org_adjudica")
                cod_org_adjudica.text = getadjudicador(str(row["NIF del Organismo Adjudicador:"])) if getadjudicador(str(row["NIF del Organismo Adjudicador:"]))!= False else v[2]
                cod_adjudicatario = ET.SubElement(contrato,"cod_adjudicatario")
                cod_adjudicatario.text = getadjudicatario(str(row["CIF/NIF Adjudicatario"]))
                num_contratos += 1 
            print(df.shape)
            
        
tree = ET.ElementTree(root_contratos)
print(os.path.join(fichero_padre,"data_blp/contratos.xml"))
tree.write(os.path.join(fichero_padre,"data_blp/contratos.xml"), encoding = "UTF-8", xml_declaration=True)


OLULA DEL RÍO-JOG


C:\Users\bllanos\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


3,NaN,Código Expte de Contratación/Encargo,Organismo Adjudicador del Contrato:,NIF del Organismo Adjudicador:,CIF/NIF Adjudicatario,Empresa Adjudicataria,Provincia Adjudicatario,Municipio Adjudicatario,Denominación del Contrato/Encargo,Objeto Contrato/Encargo,Concepto de gasto (seleccionar de la lista),Fecha (dd/mm/aaaa),"Medio (BOE, DOUE, BDNS, Perfil Contratante, etc.)",Presupuesto de la Licitación (con IVA),Presupuesto de la Licitación (sin IVA),Importe Adjudicado en la licitación (con IVA),Importe Adjudicado en la licitación (sin IVA),Importe de ayuda del Fondo de Solidaridad que se solicita (con IVA)**,Importe de ayuda del Fondo de Solidaridad que se solicita (sin IVA)**,Importe que ha sido financiado por otras vías (públicas o privadas),Fecha firma contrato/Encargo,Fecha Fin Contrato/Encargo,Duración contrato/Encargo (meses),"Tipo Contrato (Servicios, suministros, etc.)/Encargo","Tipo Procedimiento (Abierto, Negociado con/sin publicidad, Acuerdo Marco, Contrato Menor)","Tipo Tramitación (Ordinaria, Urgencia, Emergencia)",Fecha Acuerdo Órgano Contratación,Fecha inicio ejecución prestaciones,Fecha publicación formalización,OBSERVACIONES,elegible
0,NaN,T-2019/00001786_1,AYUNTAMIENTO DE OLULA DEL RÍO,P0406900A,A78139755,"Gestión y Técnicas del Agua, S.A.",Madrid,Madrid,Instalación Red Abastecimiento,Instalación Red Abastecimiento PVC-OØ90 C/ Mer...,a) Restablecimiento del funcionamiento de las ...,NaN,NaN,8063.92,6664.40,8063.92,6664.40,8063.92,6664.40,0,2019-11-01 00:00:00,2019-11-30 00:00:00,1,obras,contrato menor,ordinaria,NaN,NaN,NaN,NaN,NaN
1,NaN,T-2019/00002016_1,AYUNTAMIENTO DE OLULA DEL RÍO,P0406900A,A78139755,"Gestión y Técnicas del Agua, S.A.",Madrid,Madrid,Instalación Red Abastecimiento,Instalación Red Abastecimiento PVC-OØ200 C/ Cá...,a) Restablecimiento del funcionamiento de las ...,NaN,NaN,15256.19,12608.42,15256.19,12608.42,9476.08,7831.47,0,2019-12-01 00:00:00,2019-12-31 00:00:00,1,obras,contrato menor,ordinaria,NaN,NaN,NaN,NaN,NaN


(2, 31)
P:/Regulacion/49 Encargo MinHac 20-22/08 FSUE-DANA/02 Archivos de trabajo/BENEFICIARIOS FSUE DANA\data_blp/contratos.xml
Wall time: 1.08 s


In [30]:
empty

[]

In [31]:
error_textos

0

In [32]:
def getcontrato(ayto,exp):
    contrats = pd.read_csv(os.path.join(fichero_padre,"data_blp/correspcontratos.csv"), sep = ";", names = ["ayto", "contrato", "codigo"], encoding="latin1")
    #display(contrats)
    contrats.contrato = contrats.contrato.apply(lambda x : str(x).replace("\n", ""))
    contrats = contrats[contrats.ayto == ayto]
    #display(contrats)
    contrats = contrats[contrats.contrato == exp]
    
    if contrats.shape[0]!=1:
        print(ayto,exp)
        #display(contrats)
    return contrats.iloc[0]['codigo']
        

In [33]:
root_modops = ET.Element("operacionesModif")
for k,v in ayuntamientos.items():
    if k not in sin_fichero and k not in empty:
        #print(k)
        df = get_df_contratos(os.path.join(fichero_padre,k,"Justificacion_FSUE-DANA.xlsx"))
        contratos_asignar = df["Código Expte de Contratación/Encargo"].unique()
        #print(contratos_asignar)
        operacion = ET.SubElement(root_modops,"operacion")
        cod_poperativo = ET.SubElement(operacion,"cod_poperativo")
        cod_poperativo.text = "SO-DANA"
        cod_operacion = ET.SubElement(operacion,"cod_operacion")
        cod_operacion.text = v[0]
        contratos_pub = ET.SubElement(operacion,"contratos_publicos")
        contratos_pub.text = "S"
        contratos = ET.SubElement(operacion,"contratos")
        for item in contratos_asignar:
            contrato = ET.SubElement(contratos,"contrato")
            cod_contrato =  ET.SubElement(contrato,"cod_contrato")
            cod_contrato.text = getcontrato(k,str(item).replace("\n",""))
            
tree = ET.ElementTree(root_modops)
tree.write(os.path.join(fichero_padre,"data_blp/asignacion_contratos.xml"), encoding = "UTF-8", xml_declaration=True)     

C:\Users\bllanos\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [34]:
encomiendas = []
for k,v in ayuntamientos.items():
    if k not in sin_fichero:
        df=get_df_contratos(os.path.join(fichero_padre,k,"Justificacion_FSUE-DANA.xlsx"))
        print(k)
        if df.shape[0] == 0:
            #empty.append(k)
            print(f"{k} no tiene contratos")
        else:
            num_contrato = 0
            df.fillna('', inplace=True)
            for index, value in df.iterrows():
                num_contrato +=1
                if value["CIF/NIF Adjudicatario"].strip() == "A28476208" or value["CIF/NIF Adjudicatario"].strip() == "A79365821":
                    encomiendas.append((k, num_contrato))
                elif value["Tipo Procedimiento (Abierto, Negociado con/sin publicidad, Acuerdo Marco, Contrato Menor)"] != "nan" and "encomienda" in value["Tipo Procedimiento (Abierto, Negociado con/sin publicidad, Acuerdo Marco, Contrato Menor)"].casefold().lower():
                    encomiendas.append((k, num_contrato))
                elif "encargo" in value["Tipo Contrato (Servicios, suministros, etc.)/Encargo"].casefold().lower():
                    encomiendas.append((k, num_contrato))
            

OLULA DEL RÍO-JOG


In [35]:
encomiendas

[]